# Regression analysis - Ordinary Least Squares (OLS)

## Overview

- Regression tests/maps relationships between variables.
- The variable that is being explained (on the left) is the dependent variable; variables on the right are independent variables (or predictors)
- OLS with one independent variable is called simple OLS, more than one independent variables is called multiple OLS
- It’s a common practice to denote the outputs with 𝑦 and the inputs with 𝑥. If there are two or more independent variables, then they can be represented as the vector 𝐱 = (𝑥₁, …, 𝑥ᵣ), where 𝑟 is the number of inputs.
- So, we use regression to answer how __several variables__ are related. It is also useful for forecasting/predicting

Equation for univariate linear regression (simple OLS): y = 𝛽0 + 𝛽1𝑥 + ε<br>

Equation for multivariate linear regression (multiple OLS): y = 𝛽0 +𝛽1𝑥1 +𝛽2𝑥2 +...+ 𝛽𝑛𝑥𝑛 + ε<br>
    
#### Evaluation

There are several ways to evaluate a model:
- the t-statistic (=coefficient / standard error) and p-value of the independent variables 
- R-squared of the whole model (all independent variables combined): variance explained / total variance
- Mean-square error (MSE): the average of the error terms squared

There are many other metrics for regression, although these are the most commonly used. You can see the full list of regression metrics supported by the scikit-learn Python machine learning library here:

https://towardsdatascience.com/ways-to-evaluate-regression-models-77a3ff45ba70

#### OLS Assumptions
    
There are several assumptions that need to be satisfied for the OLS regression output to be reliable. 
The most important assumptions are (1) that the model is complete (no missing variables), and (2) the relationship is linear, and (3) that the independent variables are not correlated with the error term.

OLS is used if the dependent variable is continuous. If it is binary (zero or 1), you would need logisitic regression or probit.

## statsmodel OLS and scikit linear regression

There are two main packages that can do OLS in Python: Statsmodel and Scikit-learn (sklearn). 

The difference between the two (from https://stats.stackexchange.com/questions/146804/difference-between-statsmodel-ols-and-scikit-linear-regression)

> Statsmodels follows largely the traditional model where we want to know how well a given model fits the data, and what variables "explain" or affect the outcome, or what the size of the effect is. Scikit-learn follows the machine learning tradition where the main supported task is chosing the "best" model for prediction.

> As a consequence, the emphasis in the supporting features of statsmodels is in analysing the training data which includes hypothesis tests and goodness-of-fit measures, while the emphasis in the supporting infrastructure in scikit-learn is on model selection for out-of-sample prediction and therefore cross-validation on "test data".

    
    
        

## Example

Let's do an OLS regression in both packages. Let's do a regression to explain the market-to-book ratio with profitability (return on assets) and firm size as the independent variables.

The market to book ratio (stock price divided by the per share book value of equity) tells us how much investors are willing to pay for each $1 in book value of equity. Book value of equity is the sum of capital raised for issuing shares plus retained earnings (profits minus distributions), treasury stock (stock buybacks) are deducted.

So we will measure if the market value is driven by profitability and/or firm size (or both of course).

#### sample dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

#df = pd.read_excel(r'..\datasets\Compustat-Funda.xlsx',nrows= 1000)
df = pd.read_excel(r'..\datasets\Compustat-Funda.xlsx')

In [ ]:
# filter: only keep if positive price, equity, assets and csho (number of shares outstanding)
df = df[ ( df['prcc_f'] > 0) & ( df['ceq'] > 0) & ( df['at'] > 0) & (df['csho'] > 0 ) ]

# add mtb, roa and size
df['mtb'] = df['prcc_f'] * df['csho'] / df['ceq']
df['roa'] = df['ni'] / df['at']
df['size'] = np.log ( df['prcc_f'] * df['csho'] )

# drop if any of these have missing values
df = df.dropna(subset=['mtb', 'roa', 'size'])
df.head()

In [ ]:
df['mtb'].describe()

### OLS with Sklearn

In [ ]:
lm = LinearRegression()
lm

In [ ]:
x = df[['roa', 'size']]
y = df['mtb']
lm.fit(x, y)

In [ ]:
print('intercept', lm.intercept_, 'coefficients', lm.coef_)

In [ ]:
# fitted values
yhat = lm.predict( x )
# note that the fitted (or predicted) values are very large
yhat

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
# model performance
print('R-squared:', r2_score(y, yhat) )
print('MSE:', mean_squared_error(y, yhat)) 

In [ ]:
# R-squared by hand: it is 1 minus the % variance explained
# so, 1 - sum square residuals / variance in y
# code from: https://stackoverflow.com/questions/42033720/python-sklearn-multiple-linear-regression-display-r-squared
SS_Residual = sum((y-yhat)**2)       
SS_Total = sum((y-np.mean(y))**2)     
r_squared = 1 - (float(SS_Residual))/SS_Total
r_squared

#### Why is the R-squared so low, and why are the fitted values so large?

> We needed to winsorize the data! Outliers have a very large influence on coefficients because in OLS the sum of squared residuals is being minimized

In [ ]:
from scipy.stats import mstats 

# add winsorized variables to dataframe
df['mtb_w'] = mstats.winsorize(df['mtb'], limits=[0.01, 0.01]).data
df['roa_w'] = mstats.winsorize(df['roa'], limits=[0.01, 0.01]).data
df['size_w'] = mstats.winsorize(df['size'], limits=[0.01, 0.01]).data

x = df[['roa_w', 'size_w']]
y = df['mtb_w']
lm.fit(x, y)

In [ ]:
print('intercept', lm.intercept_, 'coefficients', lm.coef_)

In [ ]:
# model performance
print('R-squared:', r2_score(y, lm.predict( x )) )
print('MSE:', mean_squared_error(y, lm.predict( x ))) 

### OLS with Statsmodels

In [ ]:
import statsmodels.formula.api as smf
lm = smf.ols("mtb_w ~ roa_w + size_w", data=df).fit() 
lm

In [ ]:
print(lm.params)
print('R-squared', lm.rsquared)

In [ ]:
print(str(lm.summary()))

See here for an overview of properties that are available after a regression is estimated: https://tedboy.github.io/statsmodels_doc/generated/generated/statsmodels.regression.linear_model.RegressionResults.html
        
For example: mse_resid - Mean squared error of the residuals. The sum of squared residuals divided by the residual degrees of freedom.

In [ ]:
lm.mse_resid

## Training vs Test Set

Depending on the setting, sometimes the regression is estimated using the full sample. In that case all observations are used to train the model.

It is also possible to split the sample into a training dataset and a testing dataset. The training sample is used to estimate the model, and the model is evaluated by comparing the predicted values on the test set with the actual values. 

For documentation of sklearn's train_test_split, see https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
from sklearn.model_selection import train_test_split

x = df[['roa_w', 'size_w']]
y = df['mtb_w']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.40, random_state=5)

In [ ]:
df2 = pd.DataFrame(np.random.randn( len(df), 1))
df2.head()

#### Splitting the sample by hand

In [ ]:
# create train and test sample by hand
# msk is a list of booleans (True, False) with the same length as the df
# 60% of the random numbers will be True (less than 0.6)
msk = np.random.rand(len(df)) < 0.6
print( len(msk), msk[0:20] )

In [ ]:
# map all the True values into the training sample
x_train = x[ msk ]
y_train = y[ msk ]
# the remainder into the test sample (~ means Not)
x_test = x[ ~msk ]
y_test = y[ ~msk ]
print('#obs in training sample', len(x_train), '#obs in test sample:', len(x_test))

In [ ]:
# fit the model with the training data
lm = LinearRegression()
lm.fit(x_train, y_train)
lm.predict( x )
yhat_test = lm.predict(x_test)

print('R-squared:', r2_score(y_test, yhat_test) )
print('MSE:', mean_squared_error(y_test, yhat_test) ) 

Also see: 
    
- Sklearn - Linear Regression Example - https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html
- Getting r-squared from sklearn: https://stackoverflow.com/questions/42033720/python-sklearn-multiple-linear-regression-display-r-squared